### Load the neural network

In [28]:
import torch
from torch import nn

In [29]:
max_gen1 = 95  # Maximum generation capacity for Gen1
max_gen2 = 95  # Maximum generation capacity for Gen2
min_gen1 = 70  # Minimum generation capacity for Gen1
min_gen2 = 70  # Minimum generation capacity for Gen2

In [30]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # Define only the layers that will learn weights
        self.layer1 = nn.Linear(3, 10)  # 3 input features, 10 hidden units
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(10, 2)  # Only 2 outputs will have learned parameters

    def forward(self, x):
        # Pass input through the learning layers
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)

        output = x

        # # Prepare the output tensor with additional slots for manual outputs
        # # Assuming x.shape[0] is the batch size
        # output = torch.zeros(x.shape[0], 6, device=x.device)
        # output[:, 0:2] = x  # Fill learned outputs

        # # Manually compute the additional outputs
        # output[:, 2] = max_gen1 - output[:, 0]  # Difference between max possible generation and actual Gen1
        # output[:, 3] = output[:, 0] - min_gen1 # Difference between min possible generation and actual Gen1
        # output[:, 4] = max_gen2 - output[:, 1]  # Difference between max possible generation and actual Gen2
        # output[:, 5] = output[:, 1] - min_gen2 # Difference between min possible generation and actual Gen2

        return output

In [31]:
# Path to the saved model
model_path = "model.pth"

# Load the model
model = NeuralNet()
model.load_state_dict(torch.load(model_path))

print("Model loaded successfully.")

Model loaded successfully.


In [32]:
# test the output of the neural network

some_input = torch.tensor([[50.0, 40.0, 75.0]])
output = model(some_input)
print("Model output:", output)

Model output: tensor([[67.5555, 62.2476]], grad_fn=<AddmmBackward0>)


### Export the model to ONNX

In [33]:
import torch
import torch.onnx
import tempfile

# Assuming model is the instance of SimpleModel you defined and trained previously

# Model input for exporting (batch size of 10, each with 1 feature)
x_temp = some_input

# Export the model
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    #export neural network to ONNX
    torch.onnx.export(
        model,
        x_temp,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )

### Load the model to OMLT

In [34]:
import numpy as np

epsilon_infinity = 10
center = np.array([50, 40, 75])

lb = center - epsilon_infinity
ub = center + epsilon_infinity

input_bounds = {}

for i in range(3):
    input_bounds[i] = (float(lb[i]), float(ub[i])) 

In [35]:
from omlt import OmltBlock
from omlt.neuralnet import FullSpaceNNFormulation
from omlt.io import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

write_onnx_model_with_bounds(f.name, None, input_bounds)

#load the network definition from the ONNX model
print(f.name)
network_definition = load_onnx_neural_network_with_bounds(f.name)
formulation = FullSpaceNNFormulation(network_definition)

C:\Users\HP\AppData\Local\Temp\tmpg5rghg69.onnx


### Model the optimization problem with Pyomo

In [36]:
l1_norm_ball = False
l2_norm_ball = False

power_balance = True 
pb_ub_condition = True

grid_search = False 

### Power Balance Objective

The power balance objective involves analyzing the difference between generated power and the demand for power. This difference can be used to assess the stability and efficiency of a power system. Here are the steps to calculate and interpret this difference:

1. **Compute the Balance Difference**:
   - Calculate the difference between generation and demand: \( \text{generation} - \text{demand} \).

2. **Establish Upper and Lower Bounds**:
   - Define upper and lower bounds for acceptable balance differences to ensure operational safety and efficiency.

3. **Check Balance Within Bounds**:
   - Determine if the computed balance difference lies within the defined bounds.

#### Problem Analysis

- **Problem 1: Excessive Generation**
  - Determine the highest positive value the difference can take (where generation exceeds demand).
  - Check if this value is less than the upper bound margin.
  - If the value exceeds the upper bound, it indicates that generation significantly surpasses demand, which might lead to wasted resources or operational issues.

- **Problem 2: Insufficient Generation**
  - Identify the lowest negative value the difference can take (where demand exceeds generation).
  - Check if this value is greater than the lower bound margin.
  - If the value is less than the lower bound, it suggests that demand significantly surpasses generation, risking power outages or system strain.

#### Risk Implications

- **Over-Generation Risk**:
  - If the balance for Problem 1 is positive but negative for Problem 2, it indicates a risk of over-generation without a risk of unmet demand.

- **Under-Generation Risk**:
  - Conversely, if the balance for Problem 1 is negative but positive for Problem 2, it highlights a potential for unmet demand without excessive generation.

By monitoring these metrics, operators can make informed decisions to adjust generation levels or plan capacity enhancements, thereby ensuring a reliable power supply.


In [37]:
import pyomo.environ as pyo

# Assume m is your Pyomo model
m = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m.nn = OmltBlock()
m.nn.build_formulation(formulation) 

# Assuming you have already defined the variables `input` and `output` appropriately:
# For example:
m.input = pyo.Var([0, 1, 2], domain=pyo.Reals)
m.output = pyo.Var([0, 1], domain=pyo.Reals)

WARNING (W1002): Setting Var 'nn.scaled_inputs[0]' to a numeric value `0`
outside the bounds (40.0, 60.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[0]' to a numeric value `0`
outside the bounds (40.0, 60.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[1]' to a numeric value `0`
outside the bounds (30.0, 50.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[2]' to a numeric value `0`
outside the bounds (65.0, 85.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002


In [38]:
# Number of neurons in the input layer (set this to your specific size)
input_neurons = 3

# Variable representing the auxiliary variable for L1 norm
if l1_norm_ball:
    m.auxiliary = pyo.Var(range(input_neurons), domain=pyo.NonNegativeReals)

In [39]:
# Connect Pyomo model input and output to the neural network
# This essentially models the forward pass of the neural network as a constraint

@m.Constraint()
def connect_input1(mdl):
    return mdl.input[0] == mdl.nn.inputs[0]

@m.Constraint()
def connect_input2(mdl):
    return mdl.input[1] == mdl.nn.inputs[1]

@m.Constraint()
def connect_input3(mdl):
    return mdl.input[2] == mdl.nn.inputs[2]

@m.Constraint()
def connect_output1(mdl):
    return mdl.output[0] == mdl.nn.outputs[0]

@m.Constraint()
def connect_output2(mdl):
    return mdl.output[1] == mdl.nn.outputs[1]

In [40]:
type(formulation)

omlt.neuralnet.nn_formulation.FullSpaceNNFormulation

In [41]:


# Demand values for the middle constraint adjustments
demand_middle = [50, 40, 75]

if l1_norm_ball:

    # Constraint for the L1 norm of the input
    def l1_norm_constraint(m):
        return sum((m.auxiliary[i]) for i in range(input_neurons)) <= 10

    m.l1_norm = pyo.Constraint(rule=l1_norm_constraint)

    # ConstraintList to hold the symmetric bounds around the demand values
    m.u_constraints = pyo.ConstraintList()

    # Adding symmetric bounds constraints
    for i in range(input_neurons):
        m.u_constraints.add(m.auxiliary[i] >= m.input[i] - demand_middle[i])
        m.u_constraints.add(m.auxiliary[i] >= -(m.input[i] - demand_middle[i]))



In [42]:
if l2_norm_ball:

    # Constraint for the L2 norm of the input

    def l2_norm_constraint(m):
        sum_differences = 0
        for i in range(input_neurons):
            sum_differences += (m.input[i] - demand_middle[i]) ** 2
        return sum_differences <= 10 ** 2
    
    m.l2_norm = pyo.Constraint(rule=l2_norm_constraint)

In [43]:
if power_balance:
    balance = m.output[0] + m.output[1] - m.input[0] - m.input[1] - m.input[2]
    # ub_power_balance = max(10**(-2), 10**(-2) * abs(m.input[0].value + m.input[1].value + m.input[2].value))
    d_highest = max(center)
    ub_power_balance = max(10**(-2), 10**(-2) * (d_highest))
    lb_power_balance = -ub_power_balance

    print("The margin of error in power balance is: ", ub_power_balance)

    if pb_ub_condition:
        expr = ub_power_balance - balance
    else:
        expr = balance - lb_power_balance
else:
    expr = max_gen1 - m.output[0]

m.obj = pyo.Objective(expr=expr, sense=pyo.minimize)

The margin of error in power balance is:  0.75


In [44]:
# Solve the model using IPOPT
solver_status = pyo.SolverFactory('cbc').solve(m, tee=False)

for i in range(3):
    print(f"Input {i+1}: {m.input[i].value}")

for i in range(2):
    print(f"Output {i+1}: {m.output[i].value}")

print(f"Objective: {m.obj()}")

Input 1: 40.0
Input 2: 30.0
Input 3: 85.0
Output 1: 69.760808
Output 2: 63.887096
Objective: 22.102096000000017


In [45]:
# as a reminder, print the center of the ball

print(f"Center: {center}")

Center: [50 40 75]


In [46]:
# verify the norm ball constraint

if l2_norm_ball:
    # Compute the L2 norm of the input
    l2_norm = np.linalg.norm(np.array([m.input[i].value for i in range(3)]) - np.array(demand_middle), 2)
    print(f"L2 norm of the input: {l2_norm}")

In [47]:
some_input_new = torch.tensor([[m.input[0].value, 50, m.input[2].value]])
output_new = model(some_input_new)
print("Model output:", output_new)

Model output: tensor([[71.4780, 70.3608]], grad_fn=<AddmmBackward0>)


### verify the results 

In [48]:
if power_balance == False and l1_norm_ball == True:    
    outputs_1, outputs_2, outputs_3 = [], [], []
    load1_values = list(range(40, 61))
    load2_values = list(range(30, 51))
    load3_values = list(range(70, 81))

    if l1_norm_ball:
        for load1 in load1_values:
            input_demand = torch.tensor([[float(load1), 40, 75]])
            output = model(input_demand)
            objective = max_gen1 - output[0][0]
            outputs_1.append(objective.item())

        for load2 in load2_values:
            input_demand = torch.tensor([[50, float(load2), 75]])
            output = model(input_demand)
            objective = max_gen1 - output[0][0]
            outputs_2.append(objective.item())

        for load3 in load3_values:
            input_demand = torch.tensor([[50, 40, float(load3)]])
            output = model(input_demand)
            objective = max_gen1 - output[0][0]
            outputs_3.append(objective.item())


In [49]:
if power_balance == False and l1_norm_ball == True:

    # plot on the same figure the objective values for different valuees of load1, load2, and load3

    import matplotlib.pyplot as plt

    if l1_norm_ball:

        plt.plot(load1_values, outputs_1, label='Load1')
        plt.plot(load2_values, outputs_2, label='Load2')
        plt.plot(load3_values, outputs_3, label='Load3')

        plt.hlines(0, 40, 60, colors='r', linestyles='dashed')

        plt.xlabel('Load Value')
        plt.ylabel('Objective Value')
        plt.legend()
        plt.show()

In [50]:
if power_balance == False and l1_norm_ball == True:

    # plot on the same figure the objective values for different valuees of load1, load2, and load3

    import matplotlib.pyplot as plt

    if l1_norm_ball:

        plt.plot(load1_values, outputs_1, label='Load1')
        plt.plot(load2_values, outputs_2, label='Load2')
        plt.plot(load3_values, outputs_3, label='Load3')

        plt.hlines(0, 40, 60, colors='r', linestyles='dashed')

        plt.xlabel('Load Value')
        plt.ylabel('Objective Value')
        plt.legend()
        plt.show()

In [51]:
# print the minimum values for each case
if power_balance == False and l1_norm_ball == True:
    min_output_1 = min(outputs_1)
    min_output_2 = min(outputs_2)
    min_output_3 = min(outputs_3)

    print(f"Minimum objective value for Load1: {min_output_1}")
    print(f"Minimum objective value for Load2: {min_output_2}")
    print(f"Minimum objective value for Load3: {min_output_3}")

### try grid search for minimum

In [52]:
if grid_search:    
    grid_search_results = {}

    for load1 in load1_values:
        for load2 in load2_values:
            for load3 in load3_values:
                load = torch.tensor([[float(load1), float(load2), float(load3)]])
                demand_middle_tensor = torch.tensor([[50, 40, 75]])
                if l1_norm_ball:
                    abs_difference = torch.abs(load - demand_middle_tensor)
                    norm = abs_difference.sum()
                else:
                    squared_difference = (load - demand_middle_tensor) ** 2
                    norm = torch.sqrt(squared_difference.sum())
                    
                if norm <= 10:
                    output = model(load)
                    objective = max_gen1 - output[0][0]
                    grid_search_results[(load1, load2, load3)] = objective.item()

In [53]:
if grid_search:
    import torch

    grid_search_results = {}
    load1 = min(load1_values)
    load2 = min(load2_values)
    load3 = min(load3_values)

    load1_max = max(load1_values)
    load2_max = max(load2_values)
    load3_max = max(load3_values)

    demand_middle_tensor = torch.tensor([[50, 40, 75]])

    while load1 <= load1_max:
        load2 = min(load2_values)
        while load2 <= load2_max:
            load3 = min(load3_values)
            while load3 <= load3_max:
                load = torch.tensor([[float(load1), float(load2), float(load3)]])

                if l1_norm_ball:
                    abs_difference = torch.abs(load - demand_middle_tensor)
                    norm = abs_difference.sum()
                else:
                    squared_difference = (load - demand_middle_tensor) ** 2
                    norm = torch.sqrt(squared_difference.sum())
                    
                if norm <= 10:
                    output = model(load)
                    objective = max_gen1 - output[0][0]
                    grid_search_results[(load1, load2, load3)] = objective.item()

                load3 += 0.1
            load2 += 0.1
        load1 += 0.1


In [54]:
if grid_search:

    min_objective = min(grid_search_results.values())
    min_loads = [loads for loads, objective in grid_search_results.items() if objective == min_objective]

    print(f"Minimum objective value: {min_objective}")
    print(f"Corresponding loads: {min_loads}")